<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [365]:
import pandas as pd
import psycopg2

In [366]:
DBNAME = 'project_sql'
USER = 'skillfactory'
PASSWORD = 'cCkxxLVrDE8EbvjueeMedPKt'
HOST = '84.201.134.129'
PORT = 5432

In [427]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [368]:
# текст запроса
query_3_1 = f'''select COUNT(DISTINCT id)
                from public.vacancies
'''
ans_3_1 = pd.read_sql_query(query_3_1, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\567917250.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [369]:
int(ans_3_1['count'])

49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [370]:
query_3_2 = f'''select COUNT(DISTINCT id)
                from public.employers'''
ans_3_2 = pd.read_sql_query(query_3_2, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\875170478.py:3: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [371]:
int(ans_3_2['count'])

23501

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [372]:
query_3_3 = f'''select COUNT(DISTINCT id)
                from public.areas'''
ans_3_3 = pd.read_sql_query(query_3_3, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\1746627259.py:3: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [373]:
int(ans_3_3['count'])

1362

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [374]:
query_3_4 = f'''select COUNT(DISTINCT id)
                from public.industries'''
ans_3_4 = pd.read_sql_query(query_3_4, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\4153599737.py:3: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [375]:
int(ans_3_4['count'])

294

***

Сложно делать какие-то качественные выводы о данных, но зато можно оценить их примерный порядок. Речь идёт о сотнях видов деятельности, тысячах регионов(городов), десятках тысяч вакансий и работодателей, причём у многих работодателей открыто более одной вакансии.

# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [376]:
query_4_1 = f'''SELECT a.name "Регион", COUNT(*) "Количество вакансий"
FROM public.areas a 
JOIN public.vacancies v ON a.id = v.area_id
GROUP BY 1
ORDER BY 2 DESC
'''
ans_4_1 = pd.read_sql_query(query_4_1, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\2524141807.py:7: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [377]:
ans_4_1

,Регион,Количество вакансий
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


1*. Напишем запрос, который позволит узнать среднюю зарплату по регионам. Возьмём тем регионы, в которых более 30 вакансий, чтобы не судить о заработных платах в регионе через призму маленькой выборки.

In [378]:
query_dop_1 = '''SELECT a.name, AVG(coalesce((v.salary_from + v.salary_to)/2, v.salary_from, v.salary_to)), COUNT(v.id)
FROM public.vacancies v
JOIN public.areas a ON v.area_id = a.id
GROUP BY 1 
HAVING AVG(coalesce((v.salary_from + v.salary_to)/2, v.salary_from, v.salary_to)) IS NOT NULL AND COUNT(v.id) > 30
ORDER BY 2 DESC, 3 DESC'''
ans_dop_1 = pd.read_sql_query(query_dop_1, connection)
ans_dop_1

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\2927108747.py:7: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,avg,count
0,Польша,257155.666667,53
1,Турция,235435.421053,39
2,Кипр,223735.589744,136
3,Армения,219569.726415,223
4,Тбилиси,207155.537500,474
...,...,...,...
132,Гродно,37822.888889,50
133,Караганда,37293.806452,102
134,Атырау,32937.735294,93
135,Могилев,32507.875000,34


Города с самыми маленькими з-п в выборке - города Казахстана и Беларуси.

2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [379]:
query_4_2 = f'''SELECT COUNT(*)
FROM public.vacancies
WHERE salary_from IS NOT NULL OR salary_to IS NOT NULL 
'''
ans_4_2 = pd.read_sql_query(query_4_2, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\485266869.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [380]:
int(ans_4_2['count'])

24073

3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [381]:
query_4_3 = f'''SELECT AVG(salary_from) low, AVG(salary_to) high
FROM public.vacancies
'''
ans_4_3 = pd.read_sql_query(query_4_3, connection)
ans_4_3['low'] = round(ans_4_3['low']).astype('int')
ans_4_3['high'] = round(ans_4_3['high']).astype('int')

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\577941185.py:4: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [382]:
ans_4_3

,low,high
0,71065,110537


4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [383]:
query_4_4 = f'''SELECT schedule "График", employment "Занятость", COUNT(*) "Количество вакансий"
FROM public.vacancies
GROUP BY 1, 2
ORDER BY 3 DESC
'''
ans_4_4 = pd.read_sql_query(query_4_4, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\1975731009.py:6: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [384]:
ans_4_4

,График,Занятость,Количество вакансий
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


4*. Найдём среднюю зарплату для каждого из указанных на графике выше сочетаний.

In [385]:
query_dop_2 = f'''SELECT schedule "График", employment "Занятость", 
AVG(coalesce((salary_from + salary_to)/2, salary_from, salary_to)) "Зарплата", COUNT(*) "Количество вакансий"
FROM public.vacancies 
GROUP BY 1, 2
ORDER BY 3 DESC
'''
ans_dop_2 = pd.read_sql_query(query_dop_2, connection)
ans_dop_2

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\2049493127.py:7: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,График,Занятость,Зарплата,Количество вакансий
0,Гибкий график,Проектная работа,174127.250000,18
1,Вахтовый метод,Проектная работа,150000.000000,2
2,Вахтовый метод,Полная занятость,123692.572491,367
3,Удаленная работа,Полная занятость,120752.866685,7802
4,Гибкий график,Полная занятость,118981.646154,1593
5,Удаленная работа,Проектная работа,92611.741176,133
6,Полный день,Проектная работа,86624.547945,141
7,Полный день,Полная занятость,81119.429250,35367
8,Сменный график,Стажировка,70500.000000,12
9,Сменный график,Проектная работа,60000.000000,1


А вот это уже очень интересно: получается, что за полноценную работу "на удалёнке" платят больше, чем за классическое сочетание "полный день-полная занятость". Также любопытно, что за работу над проектом при гибком графике платят куда больше, чем за сочение "полный день-проект" 

5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [386]:
query_4_5 = f'''SELECT experience "Опыт работы", COUNT(*) "Зарплата"
FROM public.vacancies
GROUP BY 1
ORDER BY 2'''
ans_4_5 = pd.read_sql_query(query_4_5, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\2572974512.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [387]:
ans_4_5

,Опыт работы,Зарплата
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


5*. Посмотрим на предлагаемую заработную плату в зависимости от требуемого опыта работы.

In [388]:
query_dop_3 = f'''SELECT experience AS "Опыт работы", AVG(coalesce((salary_from + salary_to)/2, salary_from, salary_to)) AS "Зарплата"
FROM public.vacancies
GROUP BY 1
ORDER BY 2'''
ans_dop_3 = pd.read_sql_query(query_dop_3, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\2472185282.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [389]:
ans_dop_3['Зарплата'] = round(ans_dop_3['Зарплата']).astype('int')
ans_dop_3

,Опыт работы,Зарплата
0,Нет опыта,40472
1,От 1 года до 3 лет,76541
2,От 3 до 6 лет,133211
3,Более 6 лет,166256


Ну здесь всё ожидаемо: с возрастанием опыта работы растёт разаботная плата.

6*. А теперь я посмотрю на средние заработные платы в зависимости от сферы деятельности.

In [429]:
query_dop_x = '''SELECT i.name "Сфера деятельности", AVG(coalesce((v.salary_from + v.salary_to)/2, v.salary_from, v.salary_to)) "Средняя зарплата в отрасли"
FROM public.industries i 
JOIN public.employers_industries e_i on e_i.industry_id = i.id 
JOIN public.employers e ON e.id = e_i.employer_id
JOIN public.vacancies v ON v.employer_id = e.id
GROUP BY 1 
HAVING AVG(coalesce((v.salary_from + v.salary_to)/2, v.salary_from, v.salary_to)) IS NOT NULL
ORDER BY 2 DESC'''
ans_dop_x = pd.read_sql_query(query_dop_x, connection)
ans_dop_x

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\563941497.py:9: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Сфера деятельности,Средняя зарплата в отрасли
0,"Агрохимия (продвижение, оптовая торговля)",381428.571429
1,Агрохимия (производство),339875.000000
2,Игорный бизнес,191904.900000
3,Корма для животных (производство),168299.827586
4,Зоомагазин,167612.903226
...,...,...
286,Фото- и видеоуслуги,40960.857143
287,Ритуальные услуги,37401.000000
288,"Архив, библиотека, искусствоведение",36857.888889
289,Переработка газа,34068.000000


Боюсь спросить, что они подразумевают под АГРОХИМИЕЙ с зарплатами в 300+ тысяч...я бы с ними не связывался. Сомневаюсь, что работодатели предлагают "продвигать" и "производить" просто безвредные удобрения за такие деньги. 

Насчёт игорного бизнеса не удивлён. 
За производство еды для животных платят в несколько раз больше, чем на производстве детского питания. Это многое говорит о нашем обществе.

***

И так, первые очертания данных, которые находятся перед нами, начинают проявляться. В целом, датасет довольно прост и логичен. Большая часть соискателей традиционно ищет полной занятости с полным днём работы.
Что меня удивило, так это заработные платы, которые предлагают работодатели. Нижняя планка в среднем находится на отметке в 71 тыс. рублей, и это при том, что средняя зарплата в России составляет порядка 60-65 тыс. рублей. Возможно, это связано с тем, что больше всего вакансий появляется в крупных городах, где заработные платы выше, чем в среднем по стране. Ещё одна возможная причина столь высоких зарплат заключается в том, что примрено на половине вакансий не заполнены поля с зарплатой. Вероятно, за этими вакансиями и скрываются низкие доходы. Так что если в будущем нам придётся заполнять эти пропуски, то уж точно не средним, и, думаю, даже не медианным значением. Скорее всего тут понадобится 25-ый квартиль или что-то похожее по порядку величины.

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [390]:
query_5_1 = f'''SELECT e.name, COUNT(*) 
FROM public.employers e
JOIN public.vacancies v ON e.id = v.employer_id
GROUP BY 1
ORDER BY 2 DESC'''
ans_5_1 = pd.read_sql_query(query_5_1, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\590680711.py:6: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [391]:
ans_5_1.head(5)

,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [392]:
query_5_2 = f'''SELECT a.name, COUNT(e.name) "Работодатели", COUNT(v.id) "Вакансии" 
FROM public.areas a
LEFT JOIN public.employers e ON a.id = e.area
LEFT JOIN public.vacancies v ON a.id = v.area_id
GROUP BY 1
ORDER BY 2 DESC'''
ans_5_2 = pd.read_sql_query(query_5_2, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\2612389458.py:7: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [393]:
ans_5_2[ans_5_2['Вакансии'] == 0]

,name,Работодатели,Вакансии
122,Россия,410,0
143,Казахстан,207,0
190,Московская область,75,0
246,Краснодарский край,19,0
247,Ростовская область,18,0
...,...,...,...
1357,Ожерелье,0,0
1358,Волоколамск,0,0
1359,Цхалтубо,0,0
1360,Верхний Баскунчак,0,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [394]:
query_5_3 = f'''SELECT e.name "Работодатели", COUNT(DISTINCT v.area_id) "Количество вакансий"
FROM public.employers e
JOIN public.vacancies v ON e.id = v.employer_id
GROUP BY 1
ORDER BY 2 DESC''' 
ans_5_3 = pd.read_sql_query(query_5_3, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\3311593934.py:6: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [395]:
ans_5_3

,Работодатели,Количество вакансий
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14761,UniSol,1
14762,UNISTORY LLC,1
14763,UNIT6,1
14764,United Distribution,1


Почитал в интернете, что контора "Поляков  Денис Иванович", которая разместила свои вакансии аж в 88 регионах - ккомпьютерные мошенники, которые обманывают несчастных российских пенсионеров и требуют за псевдоремонт компьютеров баснословные суммы. Сейчас проверим, чем они занимаются.

In [396]:
query_polyakov = '''SELECT replace(v.key_skills, '\t', ' ') "Ключевые навыки", v.schedule "График", v.employment "Занятость", v.salary_from "Минимальная зарплата", 
i.name "Сфера деятельности"
FROM public.employers e
JOIN public.vacancies v ON e.id = v.employer_id
LEFT JOIN public.employers_industries e_i ON e_i.employer_id = e.id
LEFT JOIN public.industries i ON i.id = e_i.industry_id
WHERE e.name = 'Поляков Денис Иванович' AND v.key_skills IS NOT NULL
'''
ans_polyakov = pd.read_sql_query(query_polyakov, connection)
ans_polyakov

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\2584493927.py:9: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Ключевые навыки,График,Занятость,Минимальная зарплата,Сфера деятельности
0,Техники продаж Уверенный пользователь ПК Ремон...,Сменный график,Полная занятость,60000,None
1,Техники продаж Уверенный пользователь ПК Ремон...,Сменный график,Полная занятость,60000,None
2,Навыки продаж Навыки работы с возражениями Уве...,Полный день,Полная занятость,70000,None
3,Техники продаж Уверенный пользователь ПК Ремон...,Сменный график,Полная занятость,60000,None
4,Техники продаж Уверенный пользователь ПК Ремон...,Сменный график,Полная занятость,60000,None
5,Техники продаж Уверенный пользователь ПК Ремон...,Сменный график,Полная занятость,60000,None


Как мы видим, всё сходится - сфера деятельности не указана, а первым же навыком требуют способность впарить товар. Скорее всего, это действительно мошенники. Остерегайтесь таких и, тем более, не работайте с ними! Сначала Вы обманете пенсионера, а потом случится так, что "Поляков Денис Иванович" нашутит Вас.

4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [1]:
query_5_4 = f'''SELECT COUNT(*)
FROM public.employers e
LEFT JOIN public.employers_industries e_i ON e.id = e_i.employer_id
LEFT JOIN public.industries i ON e_i.industry_id = i.id
WHERE i.name IS NULL
'''
ans_5_4 = pd.read_sql_query(query_5_4, connection)

NameError: name 'pd' is not defined

In [398]:
int(ans_5_4['count'])

8419

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [399]:
query_5_5 = f'''SELECT e.name "Компания", COUNT(*) "Количество вакансий"
FROM public.employers e
JOIN public.employers_industries e_i ON e.id = e_i.employer_id
JOIN public.industries i ON e_i.industry_id = i.id
GROUP BY 1
HAVING COUNT(*) = 4
ORDER BY 1'''
ans_5_5 = pd.read_sql_query(query_5_5, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\1633907202.py:8: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [400]:
ans_5_5

,Компания,Количество вакансий
0,101 Интернет,4
1,21vek.by,4
2,2ГИС,4
3,2К,4
4,4 пикселя +,4
...,...,...
1131,ЮРИОН,4
1132,ЮТИП Технологии,4
1133,ЯКласс,4
1134,ЯрНео,4


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [401]:
query_5_6 = f'''
SELECT COUNT(*)
FROM public.employers e
JOIN public.employers_industries e_i ON e.id = e_i.employer_id
JOIN public.industries i ON e_i.industry_id = i.id
WHERE i.name = 'Разработка программного обеспечения'
'''
ans_5_6 = pd.read_sql_query(query_5_6, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\1107678969.py:8: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [402]:
int(ans_5_6['count'])

3553

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [403]:
tables = pd.read_html('https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8')[1]


cities = tuple(tables['Город'])


query_5_7 = f'''
SELECT a.name "Регион", COUNT(v.id) "Количество вакансий"
from vacancies v
join employers e on v.employer_id = e.id
join areas a on a.id = v.area_id
WHERE e.name = 'Яндекс' AND a.name in {cities}
GROUP BY 1

UNION

select 'Total', COUNT(v.id)
from vacancies v
left join employers e on v.employer_id = e.id
left join areas a on a.id = v.area_id
where e.name = 'Яндекс'
and a.name in {cities}
group by 1
order by 2
'''
ans_5_7 = pd.read_sql_query(query_5_7, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\3958673011.py:26: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [404]:
ans_5_7

,Регион,Количество вакансий
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Уфа,26
8,Самара,26
9,Краснодар,30


***

И так, мы видим, что больше всего вакансий выставляют крупные корпорации, такие как "Яндекс", "Газпром" и т. д., причём Яндекс в болшом отрыве от всех остальных. Это может быть связано с тем, что Яндекс обладает обширными активами в совершенно разных областях, например в такси. Его вакансии есть во всех городах-миллионниках России.
У большого количества работодателей не указана сфера деятельности. Либо забыли указать, либо компания ну уж очень разностонняя, либо темнят.
Многие работодатели ищут себе разработчиков ПО. Даёшь цифровизацию!

Почитал в интернете, что оказывается контора "Поляков  Денис Иванович", которая разместила свои вакансии аж в 88 регионах - ккомпьютерные мошенники, которые обманывают несчастных российских пенсионеров и требуют за псевдоремонт компьютеров баснословные суммы.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [405]:
query_6_1 = f'''SELECT COUNT(name)
FROM public.vacancies
WHERE lower(name) LIKE '%данн%' OR lower(name) LIKE '%data%'
'''
ans_6_1 = pd.read_sql_query(query_6_1, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\3643221414.py:5: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [406]:
int(ans_6_1['count'])

1771

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [407]:
query_6_2 = '''SELECT COUNT(name)
FROM public.vacancies
WHERE (lower(name) LIKE '%data scientist%' or  lower(name) LIKE '%data science%' or lower(name) LIKE '%исследователь данных%' OR 
(lower(name) LIKE '%ml%' AND lower(name) NOT LIKE '%html%') OR lower(name) LIKE '%machine learning%' OR lower(name) LIKE '%машинн%обучен%') AND 
(lower(name) LIKE '%junior%' OR experience = 'Нет опыта' OR schedule = 'Стажировка')
'''
ans_6_2 = pd.read_sql_query(query_6_2, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\308383263.py:7: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [408]:
int(ans_6_2['count'])

51

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [409]:
query_6_3 = '''SELECT COUNT(id) 
FROM public.vacancies 
WHERE (name ILIKE('%data scientist%') or 
name ILIKE ('%data science%') or 
name ILIKE ('%исследователь данных%') OR 
(name LIKE '%ML%'  AND name NOT ILIKE('%HTML%')) OR 
name ILIKE('%machine learning%') OR 
name ILIKE('%машинн%обучен%')) 
AND key_skills IS NOT NULL 
AND (key_skills ILIKE ('%SQL%') OR key_skills ILIKE('%postgres%'))
'''
ans_6_3 = pd.read_sql_query(query_6_3, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\2606343617.py:12: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [410]:
int(ans_6_3['count'])

201

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [411]:
query_6_4 = '''SELECT COUNT(id) 
FROM public.vacancies 
WHERE (name ILIKE('%data scientist%') or 
name ILIKE ('%data science%') or 
name ILIKE ('%исследователь данных%') OR 
(name LIKE '%ML%'  AND name NOT ILIKE('%HTML%')) OR 
name ILIKE('%machine learning%') OR 
name ILIKE('%машинн%обучен%')) 
AND key_skills IS NOT NULL 
AND (key_skills ILIKE ('%Python%'))'''
ans_6_4 = pd.read_sql_query(query_6_4, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\203735329.py:11: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [412]:
int(ans_6_4['count'])

351

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [413]:
query_6_5 = '''SELECT length(key_skills) - length(replace(key_skills, CHR(9), '')) "Количество навыков", key_skills "Необходимые навыки"
FROM public.vacancies 
WHERE (name ILIKE('%data scientist%') or 
name ILIKE ('%data science%') or 
name ILIKE ('%исследователь данных%') OR 
(name LIKE '%ML%'  AND name NOT ILIKE('%HTML%')) OR 
name ILIKE('%machine learning%') OR 
name ILIKE('%машинн%обучен%'))
'''
ans_6_5 = pd.read_sql_query(query_6_5, connection)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\1293965326.py:10: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [414]:
round(ans_6_5['Количество навыков'].mean(), 2)

5.41

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [416]:
query_6_6 = '''SELECT experience "Опыт работы", AVG(coalesce((salary_from + salary_to)/2, salary_from, salary_to)) "Средняя зарплата"
FROM public.vacancies 
WHERE (name ILIKE('%data scientist%') or 
name ILIKE ('%data science%') or 
name ILIKE ('%исследователь данных%') OR 
(name LIKE '%ML%'  AND name NOT ILIKE('%HTML%')) OR 
name ILIKE('%machine learning%') OR 
name ILIKE('%машинн%обучен%'))
AND (salary_from IS NOT NULL OR salary_to IS NOT NULL)
GROUP BY 1'''
ans_6_6 = pd.read_sql_query(query_6_6, connection)
ans_6_6

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\648943439.py:11: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,Опыт работы,Средняя зарплата
0,Нет опыта,74642.857143
1,От 1 года до 3 лет,139674.750000
2,От 3 до 6 лет,243114.666667


In [417]:
import plotly.express as px
plot = px.bar(
    data_frame=ans_6_6,
    x='Опыт работы',
    y='Средняя зарплата',
    color='Опыт работы',
    text='Средняя зарплата',
    title='Столбчатая диаграмма зависимости средней зарплаты от опыта работы'
)

plot.write_html('plotly/plot9.html')
plot.show()

In [ ]:
int(round(ans_6_6[ans_6_6['experience'] == 'От 3 до 6 лет']['avg']))

243115

7*. Ищем рай для Data Scientist-а по зарплате.

In [424]:
query_data_heaven = '''SELECT a.name, v.name, AVG(coalesce((v.salary_from + v.salary_to)/2, v.salary_from, v.salary_to)), COUNT(v.id)
FROM public.vacancies v
JOIN public.areas a ON v.area_id = a.id
GROUP BY 1, 2  
HAVING (v.name ILIKE('%data scientist%') or 
v.name ILIKE ('%data science%') or 
v.name ILIKE ('%исследователь данных%') OR 
(v.name LIKE '%ML%'  AND v.name NOT ILIKE('%HTML%')) OR 
v.name ILIKE('%machine learning%') OR 
v.name ILIKE('%машинн%обучен%')) 
AND AVG(coalesce((v.salary_from + v.salary_to)/2, v.salary_from, v.salary_to)) IS NOT NULL
ORDER BY 3 DESC'''
ans_heaven = pd.read_sql_query(query_data_heaven, connection)
ans_heaven.head(1)

C:\Users\Константин\AppData\Local\Temp\ipykernel_1804\1211938373.py:13: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



,name,name,avg,count
0,Москва,ML Engineer \ Data scientist,450000.0,1


Как и ожидалось, вакансия с самой высокой в Data Science зарплатой - из Москвы

***

Первый и главный вывод - Data Scientist-ы много зарабатывают. В особенности те, кто уже имеет какой-то опыт работы. Если закончу курс, без хлеба на столе не останусь. Хотя если я буду писать на Sql с такой же скоростью, как и сейчас...но не будем о грустном. 

Знание Python ценится чаще, чем способноость управляться с SQL. При этом потециальным работникам желательно владеть большим количеством навыков - в среднем работодатели просят знания 5-6 языков/инстументов для обработки данных.
Очень мало вакансий на стажировку - всего 51. Если учесть, что всего к работе с данными относится больше полутора тысяч вакансий, то выходит всего несколько процентов. 
В итоге можно сделать вывод, что за работу в Data Science работодатели готовы платить большие деньги своим сотрудникам, но при этом они ждут от Data Scientist-ов высокой эффективности, основанной на опыте. 
"Вот такая загогулина, понимаешь!" (с)

# Общий вывод по проекту

В итоге можно сделать вывод, что за работу в Data Science работодатели готовы платить большие деньги своим сотрудникам, но при этом ждут от Data Scientist-ов профессионализма и эффективности, основанных на опыте. "Вот такая загогулина, понимаешь!" (с)

В целом, датасет даёт мне возможность сделать вывод о том, что HeadHunter-ом пользуются соискатели и работодатели, занимающиеся, зачастую, высокотехнологичными, а значит высокооплачиваемыми отраслями. Немалая их часть связана с программированием и программным обеспечением. Большая часть вакансий из России. В датасете присутствуют фирмы разного размера и разной степени законности.